Here I try to recreate all the function in the R files (estimator, filter and simulator) in the Python language to check if I have really understood the topic.

In [13]:
import math
import numpy as np

In [11]:
def martingale_diff(y, mu_t, varsigma, nu):
    u_t = ((1 / (1 + (y - mu_t)^2/(nu*varsigma)) * (y - mu_t)))

    return u_t

In [12]:
def uSTD_uDCS_t(y, mu_t, varsigma, nu, log):
    ulpdf = (lgamma((nu + 1) / 2) - lgamma(nu / 2) - (1/2) * log(varsigma) -
              (1/2)  * log(pi * nu) - ((nu + 1) / 2) * log(1 + (y - mu_t)^2 / (nu*varsigma) ))
    if (log != TRUE):
        ulpdf = math.exp(ulpdf)
        
    return ulpdf

In [17]:
def uDCS_t_model_filter (y, theta):
    
    T = len(y)

    dloglik = np.zeros(n)
    loglik = float()

    omega = theta[1]
    phi = theta[2]
    k = theta[3]
    varsigma = theta[4]
    nu = theta[5]

    mu_t = np.full(T+1, np.nan)
    u_t = np.full(T, np.nan)

    mu_t[0] = omega

    dloglik[0] = uSTD_uDCS_t(y[0], mu_t[0], varsigma=varsigma, nu=nu, log=TRUE)

    for t in range (1, T +1):
        u_t[t-1] = martingale_diff(y[t-1], mu[t-1], varsigma, nu)
        mu_t[t] = omega + phi * (mu_t[t-1]- omega) + k * u_t[t-1]

        if t < T+1:
            dloglik[t] = uSTD_uDCS_t(y[t], mu_t=mu_t[t], varsigma= varsigma, nu = nu, log= TRUE)
            loglik = loglik + dloglik[t]

    mu_t = np.array([np.nan] * (T+1))
    u_t = np.array([np.nan] * T)

    out = {'Dynamic Location': mu_t,
           'Innovation': u_t,
           'Log_Densities_i': dloglik,
           'Log_Likelihood' : loglik
    }

    return out
    


In [20]:
def uSTDT_rnd (n, mu, varsigma, nu):
    z = np.random.standard_t(n, df = nu)
    y = float()
    for i in range(1, n):
        y[i] = mu + z[i]* math.sqrt(varsigma)
    
    return y
    

In [21]:
def uDCS_t_model_simulator (T, omega, phi, k, varsigma, nu):

    y = np.full(T, np.nan)
    mu_t = np.array(T, np.nan)
    u_t = np.array(T-1, np.nan)

    mu_t[0] = omega

    y[0] = uSTDT_rnd(1, mu_t[1], varsigma, nu)

    for t in range(1, T):

        u_t[t-1] = martingale_diff(y[t-1], mu_t[t-1], varsigma, nu)
        mu_t[t] = omega + phi * (mu_t[t-1] - omega) + k * u_t[t-1]
        y[t] = uSTDT_rnd(1, mu_t[t], varsigma, nu)

    out = {
        'y_t_gen': np.array(y),
        'Dynamic_Location': np.array(mu_t),
        'Innovation_u_t': np.array(u_t)
    }

    return out

In [ ]:
def uDCS_t_model_estimator (data, param):

    T = len(data)

    omega = param[1]
    phi = param[2]
    k = param[3]

    varsigma = param[4]
    nu = param[5]

    theta_st = [omega, phi, k, varsigma, nu]

    lower =[-Inf, -0.999, -2, 1e-05, 2.099]
    upper =[ Inf,  0.999,  2, Inf, 300]